In [1]:
import shapely
import os
from patchify import patchify
from shapely import geometry
from shapely.geometry import Point, MultiPoint
import json
import slideio
import numpy as np
import cv2
import geojson
from ast import literal_eval

In [2]:
def GeoData (file):
    annotation_geojson_file = open(file)
    global collection 
    collection = geojson.load(annotation_geojson_file)

In [3]:
def ExampleTiling(path, imagetype):
    slide = slideio.open_slide(path,imagetype)
    scene = slide.get_scene(0)
    image_resX,image_resY = scene.size
    image = scene.read_block((0,0,0,0),(image_resX//10,image_resY//10))
    image2 = scene.read_block((0,0,0,0),(image_resX,image_resY))
    str_search = slide.file_path
    slide_name = str_search.split('\\')[-1].split('.')[0]
    isClosed = True
    thickness = 5
    for feature in collection["features"]:
        anno_name = str(feature['properties']['classification']["name"])
        coordinates = str(feature['geometry']['coordinates'])
        directory = (r"C:\Users\villa\Desktop\Work Files\Tiles\\" + anno_name)
        if not os.path.exists(directory):
            os.mkdir(directory)
        os.chdir(directory)
        color = (34,65,200)
        pts = (literal_eval(coordinates))
        pts = np.array(pts)
        
        makeMask = cv2.drawContours(image2, pts, -1, color=(255, 0, 0), thickness=cv2.FILLED)
        mask = np.zeros_like(makeMask)
        cv2.drawContours(mask,pts,0,(255,255,255),-1)
        maskName = directory+"\\"+anno_name+" Mask.png"
        cv2.imwrite(maskName,mask)
        maskSlide = slideio.open_slide(maskName,"GDAL")
        maskScene = maskSlide.get_scene(0)
        
        x, y, width, height = cv2.boundingRect(pts)
        area_image = scene.read_block((0,0,0,0),(image_resX//10,image_resY//10))
        for y_coord in range(y,y+height,512):
            for x_coord in range(x,x+width,512):
                testCoord = (x_coord,y_coord)
                pointTest = cv2.pointPolygonTest(pts,testCoord,False)
                if pointTest>0:
                    drawPoly = pts//10
                    coords = (x_coord,y_coord)
                    coord_str = ("X="+str(x_coord)+", Y="+str(y_coord))
                    tile_image = scene.read_block((x_coord,y_coord,512,512),(image_resX//10,image_resY//10))
                    mask_image = maskScene.read_block((x_coord,y_coord,512,512),(image_resX//10,image_resY//10))
                    cv2.imwrite(slide_name+"["+coord_str+"]"+".png",tile_image)
                    cv2.imwrite(slide_name+"["+coord_str+"]"+" Mask.png",mask_image)
                    cv2.polylines(area_image,[drawPoly],isClosed,color,thickness)
                    cv2.circle(area_image,(x_coord//10,y_coord//10),radius = 5,color = (255,0,0),thickness = -1)
                else:
                    pass
        cv2.imwrite("area Covered.png",area_image)
    dicPath = slide.file_path.split("CZI")[0]

In [4]:
GeoData(r"C:\Users\villa\Desktop\Work Files\QUpath Example for PaQuo forum\QuPath deidentified\export\file.geojson") 
ExampleTiling(r"C:\Users\villa\Desktop\Work Files\QUpath Example for PaQuo forum\CZI\HE\MLC 1__HE_.czi","CZI")